In [68]:
# Import libraries necesarry for web scraping with selenium
#import selenium
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service as ChromeService
#import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

import requests
from bs4 import BeautifulSoup
import json
import PyPDF2
import io
import re
import pandas as pd
# Import necessary libraries


Here are the Websites I am thinking of using:

https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/docApi

https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/topic-pages/tables/table-5

https://www.icpsr.umich.edu/web/NACJD/studies/38604/summary

https://bjs.ojp.gov/library/publications/criminal-victimization-2022

https://endrapeoncampus.org/map-and-tool/

https://ope.ed.gov/campussafety/#/compare/confirm

https://www.prri.org/research/2020-census-of-american-religion/

https://www.statista.com/statistics/221454/share-of-religious-americans-by-state/

BIG 12 (for 2024 group) FIRE & SAFETY REPORTS:

BYU https://police.byu.edu/2023-asr-provo  (page 9)

Baylor https://www.baylor.edu/risk/doc.php/399779.pdf (page 12)

UCF https://police.ucf.edu/sites/default/files/2023%20Annual%20Security%20%26%20Fire%20Safety%20Report.pdf (page 78)

Cinncinati https://www.uc.edu/content/dam/refresh/publicsafety-62/docs/ucpd/2023-annual-security-report.pdf (page 34)

Houston http://m2s-conf.uh.edu/police/records-reports/annualreport/2023-uh-annual-security-and-fire-safety-report.pdf (page 39)

Iowa https://www.police.iastate.edu/wp-content/uploads/2023/09/2022-Report-Final.pdf (page 6)

KU https://civilrights.ku.edu/sites/civilrights/files/documents/Reports/University%20of%20Kansas%20Annual%20Security%20and%20Fire%20Safety%20Report-2023-KLETC.pdf (page 60)

K-state https://www.k-state.edu/report/clery/reports/KSUCleryReport2023.pdf (page 13)

OSU https://safety.okstate.edu/police/documents/annual-security-reports/2023-annual-safety-report.pdf (page 37)

TCU https://police.tcu.edu/wp-content/uploads/2023/09/TCU-Annual-Security-Report-Fire-Safety-Report_2023.v1.pdf (page 80)

Texas Tech https://www.depts.ttu.edu/clery/reports/2023LubbockASRfinal.pdf (page 107)

West Virginia https://police.wvu.edu/files/d/a1a0a6fe-444e-4353-8e92-7dcbeb00cd27/2023-annual-security-and-fire-safety-report-with-appendix.pdf (page 36)

In [2]:
import camelot

In [115]:
BYU_table = camelot.read_pdf('BYU_report.pdf', pages='9', flavor='stream')
Baylor_tables = camelot.read_pdf('Baylor_report.pdf', pages='12,13', flavor='stream')
Cinn_table = camelot.read_pdf('Cinn_report.pdf', pages='34', flavor='stream')
Iowa_table = camelot.read_pdf('Iowa_report.pdf', pages='6', flavor='stream')
Kstate_table = camelot.read_pdf('Kstate_report.pdf', pages='13', flavor='stream')
KU_table = camelot.read_pdf('KU_report.pdf', pages='60', flavor='stream')
OSU_table = camelot.read_pdf('OSU_report.pdf', pages='37', flavor='stream')
TCU_table = camelot.read_pdf('TCU_report.pdf', pages='80', flavor='stream')
Ttech_table = camelot.read_pdf('Ttech_report.pdf', pages='107', flavor='stream')
UCF_tables = camelot.read_pdf('UCF_report.pdf', pages='78,79,80', flavor='stream')
UH_table = camelot.read_pdf('UH_report.pdf', pages='39', flavor='stream')
WV_table = camelot.read_pdf('WV_report.pdf', pages='36', flavor='stream')

In [150]:
BYU_table[0].df
# take only rows 0,1, and 9-12
BYU_sexOff = BYU_table[0].df.iloc[[1,9,10,11,12]]
# take out columns 2,4,6
BYU_sexOff = BYU_sexOff.drop([2,4,6], axis=1)
# reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
# make the first row the column names
BYU_sexOff.columns = BYU_sexOff.iloc[0]
# drop the first row
BYU_sexOff = BYU_sexOff.drop([0])
# reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
# Add the school name
BYU_sexOff['School'] = 'BYU'
# Adding the year counts together
BYU_sexOff['2020 Total'] = BYU_sexOff['2020'].astype(int).sum(axis = 1)
BYU_sexOff['2021 Total'] = BYU_sexOff['2021'].astype(int).sum(axis = 1)
BYU_sexOff['2022 Total'] = BYU_sexOff['2022'].astype(int).sum(axis = 1)
# Keep only the columns we want
BYU_sexOff = BYU_sexOff[['Year Reported','School','2020 Total','2021 Total','2022 Total']]
# Rename Year Reported to Offense
BYU_sexOff = BYU_sexOff.rename(columns={'Year Reported': 'Offense'})

#BYU_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,BYU,4,7,6
1,Fondling,BYU,10,13,17
2,Incest,BYU,0,0,0
3,Statutory Rape,BYU,0,0,0


In [143]:
Baylor_tables[0].df
# Only take row 16 and below
Baylor_sexOff = Baylor_tables[0].df[16:]
Baylor_sexOff = Baylor_sexOff.drop([19], axis=0)
# reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)
# Remove the 0 and 2 columns
Baylor_sexOff = Baylor_sexOff.drop([0,2], axis=1)
# Add a row with custom values
data = {1:'Statutory Rape', 3:'0',4:'0',5:'0',6:'0',7:'0',8:'0'}
# Append the row to the dataframe
Baylor_sexOff = Baylor_sexOff.append(data, ignore_index=True)
# only keep columns 1,6
Baylor_sexOff = Baylor_sexOff[[1,6]] 
# Rename the columns
Baylor_sexOff = Baylor_sexOff.rename(columns={1: 'Offense', 6: '2020 Total'}) 
# remove the first row
Baylor_sexOff = Baylor_sexOff.drop([0], axis=0) 

Baylor_sexOff['School'] = 'Baylor'
Baylor_sexOff['2021 Total'] = ['20','10','0','0']
Baylor_sexOff['2022 Total'] = ['22','8','0','0']

# Reorder the columns
Baylor_sexOff = Baylor_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)
#Baylor_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\2115857788.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Baylor_sexOff = Baylor_sexOff.append(data, ignore_index=True)


,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,Baylor,13,20,22
1,Fondling,Baylor,13,10,8
2,Incest,Baylor,0,0,0
3,Statutory Rape,Baylor,0,0,0


In [171]:
Cinn_table[0].df
# Keep rows 3, 6-9
Cinn_sexOff = Cinn_table[0].df.iloc[[3,6,7,8,9]]
# Reset the index
Cinn_sexOff = Cinn_sexOff.reset_index(drop=True)
# Make first row the columns
Cinn_sexOff.columns = Cinn_sexOff.iloc[0]
# Drop the first row
Cinn_sexOff = Cinn_sexOff.drop([0])
# Reset the index
Cinn_sexOff = Cinn_sexOff.reset_index(drop=True)
# Remove all the * from row 1
Cinn_sexOff.iloc[0,] = Cinn_sexOff.iloc[0,].str.replace('*','')
# Add the school name
Cinn_sexOff['School'] = 'CU'
# Sum the columns
Cinn_sexOff['2020 Total'] = Cinn_sexOff['2020'].astype(int).sum(axis = 1)
Cinn_sexOff['2021 Total'] = Cinn_sexOff['2021'].astype(int).sum(axis = 1)
Cinn_sexOff['2022 Total'] = Cinn_sexOff['2022'].astype(int).sum(axis = 1)
# Keep only the columns we want
Cinn_sexOff = Cinn_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Swap rows 3 and 4
Cinn_sexOff.iloc[[2,3]] = Cinn_sexOff.iloc[[3,2]]
#Cinn_sexOff


C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\845297068.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Cinn_sexOff.iloc[0,] = Cinn_sexOff.iloc[0,].str.replace('*','')


,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,CU,20,34,52
1,Fondling,CU,7,20,19
2,Incest,CU,0,0,0
3,Statutory Rape,CU,0,0,0


In [176]:
Iowa_table[0].df
# Keep idex rows 0,1, 8-11
Iowa_sexOff = Iowa_table[0].df.iloc[[0,1,8,9,10,11]]
# Reset the index
Iowa_sexOff = Iowa_sexOff.reset_index(drop=True)
# Keep columns 0,16,17,18
Iowa_sexOff = Iowa_sexOff[[0,16,17,18]]
# Drop row 0 and 1
Iowa_sexOff = Iowa_sexOff.drop([0,1], axis=0)
# Reset the index
Iowa_sexOff = Iowa_sexOff.reset_index(drop=True)
# Rename the columns
Iowa_sexOff = Iowa_sexOff.rename(columns={0: 'Offense', 16: '2020 Total', 17: '2021 Total', 18: '2022 Total'})
# Add the school name
Iowa_sexOff['School'] = 'Iowa'
# Reorder the columns
Iowa_sexOff = Iowa_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]

#Iowa_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,Iowa,10,22,18
1,Fondling,Iowa,4,7,6
2,Incest,Iowa,0,0,0
3,Statutory Rape,Iowa,0,0,0


In [189]:
Kstate_table = camelot.read_pdf('Kstate_report.pdf', pages='13', flavor='stream', table_regions=['0,600,800,0'])

In [ ]:
Kstate_table[0].df
# Keep idex rows 9, 17-20
Kstate_sexOff = Kstate_table[0].df.iloc[[9,17,18,19,20]]
# Rename the columns
Kstate_sexOff.columns = ['Offense','2020','2020','2020','2020','2021','2021','2021','2021','2022','2022','2022','2022']
# Make yearly totals
Kstate_sexOff['2020 Total'] = Kstate_sexOff['2020'].astype(int).sum(axis = 1)
Kstate_sexOff['2021 Total'] = Kstate_sexOff['2021'].astype(int).sum(axis = 1)
Kstate_sexOff['2022 Total'] = Kstate_sexOff['2022'].astype(int).sum(axis = 1)
# Make school column
Kstate_sexOff['School'] = 'KSU'
# Keep only the columns we want
Kstate_sexOff = Kstate_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reset the index
Kstate_sexOff = Kstate_sexOff.reset_index(drop=True)

#Kstate_sexOff

In [222]:
KU_table[0].df
# Keep idex rows 8-19
KU_sexOff = KU_table[0].df.iloc[[8,9,10,11,12,13,14,15,16,17,18,19]]
# Reset the index
KU_sexOff = KU_sexOff.reset_index(drop=True)
# Pivot the table, keeping columns 0 the same, and moving columns 1 to the rows


#KU_sexOff

In [221]:
# Create a data frame from scratch that matches the BYU_sexOff data frame for the KU data
KU_scratch = pd.DataFrame(columns=['Offense','School','2020 Total','2021 Total','2022 Total'])
# Add this row to the data frame
KU_data = {'Offense':'Rape', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Fondling', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'1'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Incest', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Statutory Rape', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)

KU_sexOff = KU_scratch

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3252969099.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3252969099.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3252969099.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3252969099.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = K

In [271]:
OSU_table[0].df
# Keep idex rows 0 9-20
OSU_sexOff = OSU_table[0].df.iloc[[9,10,11,12,13,14,15,16,17,18,19,20]]
# Keep index columns 0,1,5
OSU_sexOff = OSU_sexOff[[0,1,5]]
# Fill in the rest of column 0
OSU_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
# Replace Column 0 with the Offense column
OSU_sexOff = OSU_sexOff[['Offense',1,5]]
# Pivot the table
OSU_sexOff = OSU_sexOff.pivot(columns=1, index='Offense')
OSU_sexOff.columns = OSU_sexOff.columns.droplevel()
OSU_sexOff = OSU_sexOff.reset_index()
OSU_sexOff = OSU_sexOff.rename_axis(None, axis=1)

# Add the school name
OSU_sexOff['School'] = 'OSU'
# Rename the columns and Reorder the columns
OSU_sexOff = OSU_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
OSU_sexOff = OSU_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reorder the rows
OSU_sexOff = OSU_sexOff.iloc[[2,0,1,3]]
# Reset the index
OSU_sexOff = OSU_sexOff.reset_index(drop=True)



OSU_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,OSU,4,15,12
1,Fondling,OSU,6,4,0
2,Incest,OSU,0,0,0
3,Statutory Rape,OSU,0,0,0


In [272]:
TCU_table[0].df

,0,1,2,3,4,5
0,Offense,,On-campus,,Building or,
1,,,,Housing Facilities,,Property
2,,,,,Property,
3,CRIMINAL HOMICIDE,,,,,
4,,2022,0,0,0,0
5,Murder and Non-negligent Manslaughter,2021,0,0,0,0
6,,2020,0,0,0,0
7,,2022,0,0,0,0
8,Manslaughter by Negligence,2021,0,0,0,0
9,,2020,0,0,0,0


In [273]:
Ttech_table[0].df

,0,1,2,3,4,5
0,,,CRIMINAL OFFENSES,,,
1,,On-Campus,On-campus,Non-campus,Public,
2,Crime Classification,,,,,Total
3,,Student Housing,Total,Property,Property,
4,Murder / Non-,,,,,
5,Negligent,0,0,0,0,0
6,Manslaughter,,,,,
7,Manslaughter by,,,,,
8,,0,0,0,0,0
9,Negligence,,,,,


In [297]:
UCF_tables[0].df

,0,1,2,3,4
0,Crimes,,,,
1,,,On Campus,,
2,,,,,Public
3,Types of Crimes,On Campus,Residential,Non-Campus,
4,,,,,Property
5,,,Facility,,
6,Murder/Non-Negligent Manslaughter,0,0,0,0
7,Manslaughter by Negligence,0,0,0,0
8,Rape,14,11,0,0
9,Fondling,6,2,2,0


In [296]:
UH_table[0].df
# Keep rows  13-26
UH_sexOff = UH_table[0].df.iloc[[14,15,16,17,18,19,20,21,22,23,24,25]]
# Keep columns 0,1,6
UH_sexOff = UH_sexOff[[0,1,6]]

UH_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
# Replace Column 0 with the Offense column
UH_sexOff = UH_sexOff[['Offense',1,6]]
# Pivot the table
UH_sexOff = UH_sexOff.pivot(columns=1, index='Offense')
UH_sexOff.columns = UH_sexOff.columns.droplevel()
UH_sexOff = UH_sexOff.reset_index()
UH_sexOff = UH_sexOff.rename_axis(None, axis=1)

# Add the school name
UH_sexOff['School'] = 'UH'
# Rename the columns and Reorder the columns
UH_sexOff = UH_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
UH_sexOff = UH_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reorder the rows
UH_sexOff = UH_sexOff.iloc[[2,0,1,3]]
# Reset the index
UH_sexOff = UH_sexOff.reset_index(drop=True)
UH_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,UH,5,12,13
1,Fondling,UH,4,5,1
2,Incest,UH,0,0,0
3,Statutory Rape,UH,1,0,0


In [291]:
WV_table[0].df 
# Keep rows 15-21
WV_sexOff = WV_table[0].df.iloc[[15,16,17,18,19,20]]
WV_sexOff.iloc[0,3] = '19'
WV_sexOff.iloc[3,2] = '7'
# Keep only columns 0,1,6
WV_sexOff = WV_sexOff[[0,1,6]]

WV_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling']
# Replace Column 0 with the Offense column
WV_sexOff = WV_sexOff[['Offense',1,6]]

# Pivot the table
WV_sexOff = WV_sexOff.pivot(columns=1, index='Offense')
WV_sexOff.columns = WV_sexOff.columns.droplevel()
WV_sexOff = WV_sexOff.reset_index()
WV_sexOff = WV_sexOff.rename_axis(None, axis=1)

# Add the school name
WV_sexOff['School'] = 'WV'
# Rename the columns and Reorder the columns
WV_sexOff = WV_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
WV_sexOff = WV_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Fill in the empty Data
WV_data = {'Offense':'Incest', 'School': 'WV', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
WV_sexOff = WV_sexOff.append(WV_data, ignore_index=True)
WV_data = {'Offense':'Statutory Rape', 'School': 'WV', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
WV_sexOff = WV_sexOff.append(WV_data, ignore_index=True)
# Reorder the rows
WV_sexOff = WV_sexOff.iloc[[1,0,2,3]]
# Reset the index
WV_sexOff = WV_sexOff.reset_index(drop=True)

WV_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3118606000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WV_sexOff.iloc[0,3] = '19'
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3118606000.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WV_sexOff.iloc[3,2] = '7'
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3118606000.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  WV_sexOff = WV_sexOff.append(WV_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3118606000.py:28: FutureWarning: The 

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,WV,9,17,21
1,Fondling,WV,4,12,17
2,Incest,WV,0,0,0
3,Statutory Rape,WV,0,0,0
